In [12]:
# Create a new set of 2,000 random latitudes and longitudes.
import numpy as np

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
coordinates = list(zip(lats, lngs))

In [13]:
# Get the nearest city using the citipy module.
from citipy import citipy
cities = [citipy.nearest_city(
    coordinate[0],
    coordinate[1]
).city_name for coordinate in coordinates]

In [1]:
# Perform an API call with the OpenWeatherMap.
import requests
from config import weather_api_key

In [ ]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)